In [5]:
import torch
import numpy as np
import torchvision
import torch.nn as nn
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

In [9]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((32, 32)),  # Resize images to 32x32 pixels
    torchvision.transforms.ToTensor(),        # Convert images to PyTorch tensors
    torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,))  # Normalize with mean=0.5, std=0.5
])

# Load the MNIST training dataset
# - root: Directory to store/download the dataset
# - train: True for training set
# - download: True to download the dataset if not present
# - transform: Apply the defined transformations
train_dataset = torchvision.datasets.Imagenette(
    root='/home/kami/Documents/datasets/',
    size= "160px",
    download=False,
    transform=transform
)

# Create a DataLoader to batch and shuffle the training data
# - batch_size: 64 for manageable batch processing
# - shuffle: True to randomize the data order
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True
)


# Example: Iterate through one batch to verify the data
for images, labels in train_loader:
    print(f"Batch shape: {images.shape}")  # Should be [64, 1, 32, 32] (batch, channels, height, width)
    print(f"Labels shape: {labels.shape}")  # Should be [64]
    print(f"Image tensor min: {images.min()}, max: {images.max()}")  # Check normalization
    # print()
    break  # Only print the first batch




Batch shape: torch.Size([64, 3, 32, 32])
Labels shape: torch.Size([64])
Image tensor min: -1.0, max: 1.0

Batch shape: torch.Size([64, 3, 32, 32])
Labels shape: torch.Size([64])
Image tensor min: -1.0, max: 1.0

Batch shape: torch.Size([64, 3, 32, 32])
Labels shape: torch.Size([64])
Image tensor min: -1.0, max: 1.0

Batch shape: torch.Size([64, 3, 32, 32])
Labels shape: torch.Size([64])
Image tensor min: -1.0, max: 1.0

Batch shape: torch.Size([64, 3, 32, 32])
Labels shape: torch.Size([64])
Image tensor min: -1.0, max: 1.0

Batch shape: torch.Size([64, 3, 32, 32])
Labels shape: torch.Size([64])
Image tensor min: -1.0, max: 1.0

Batch shape: torch.Size([64, 3, 32, 32])
Labels shape: torch.Size([64])
Image tensor min: -1.0, max: 1.0

Batch shape: torch.Size([64, 3, 32, 32])
Labels shape: torch.Size([64])
Image tensor min: -1.0, max: 1.0

Batch shape: torch.Size([64, 3, 32, 32])
Labels shape: torch.Size([64])
Image tensor min: -1.0, max: 1.0

Batch shape: torch.Size([64, 3, 32, 32])
Label

In [7]:
class LeNet5(nn.Module):
    """LeNet-5 convolutional neural network for MNIST classification."""
    def __init__(self, num_classes=10 , in_channels=1):
        super(LeNet5, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels, 6, kernel_size=5, stride=1, padding=0)  # Input: 1x32x32, Output: 6x28x28
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)  # Input: 6x14x14, Output: 16x10x10
        # Pooling layer (used twice)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)  # Downsample by 2
        # Fully connected layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # Input: 16x5x5 (after pooling), Output: 120
        self.fc2 = nn.Linear(120, 84)          # Output: 84
        self.fc3 = nn.Linear(84, num_classes)           # Output: 10 (MNIST classes)
        # Activation function
        self.relu = nn.ReLU()

    def forward(self, x):
        """Forward pass through the LeNet-5 network."""
        x = self.relu(self.conv1(x))      # Conv1 -> ReLU
        x = self.pool(x)                  # Pool (28x28 -> 14x14)
        x = self.relu(self.conv2(x))      # Conv2 -> ReLU
        x = self.pool(x)                  # Pool (10x10 -> 5x5)
        x = x.view(-1, 16 * 5 * 5)       # Flatten for FC layers
        x = self.relu(self.fc1(x))        # FC1 -> ReLU
        x = self.relu(self.fc2(x))        # FC2 -> ReLU
        x = self.fc3(x)                   # FC3 (no activation, logits for classification)
        return x


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# device = torch.device('cpu')
model = LeNet5(num_classes=10, in_channels=3).to(device)
criterion = nn.CrossEntropyLoss()  # Combines log softmax and NLL loss
optimizer = optim.Adam(model.parameters(), lr=1e-3)  # Adam with learning rate 1e-3

st = time.time()
# Train for 10 epochs
model.train()
for epoch in range(10):
    batch_index = 0
    for images, labels in train_loader:
        # Move data to device
        images = images.to(device)
        labels = labels.to(device)

        # Zero out gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Log loss every 100 batches
        batch_index += 1
        if batch_index % 100 == 0:
            print(f"Epoch: {epoch} | Batch: {batch_index} | Loss: {loss.item():.4f}")

et = time.time()
print(et-st)
print("Training completed.")

cuda
Epoch: 0 | Batch: 100 | Loss: 1.8251
Epoch: 1 | Batch: 100 | Loss: 1.7071
Epoch: 2 | Batch: 100 | Loss: 1.7536
Epoch: 3 | Batch: 100 | Loss: 1.6043
Epoch: 4 | Batch: 100 | Loss: 1.2341
Epoch: 5 | Batch: 100 | Loss: 1.3011
Epoch: 6 | Batch: 100 | Loss: 1.4577
Epoch: 7 | Batch: 100 | Loss: 1.3226
Epoch: 8 | Batch: 100 | Loss: 1.1701
Epoch: 9 | Batch: 100 | Loss: 1.2263
28.323670387268066
Training completed.
